<a href="https://colab.research.google.com/github/vaniamv/final-project-edit/blob/main/Streaming_RT_ETL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplication in Real Time to Read Carris data - group 1 - ETL approach

This notebook documents the steps to implement a data pipeline leveraging Google Cloud Platform (GCP), following an ETL (Extract, Transform, Load) approach. The pipeline processes data in three stages:

* Streaming Ingestion and Transformation (Extract and Transform): Data is ingested in real-time from a bucket that gets the vehicles endpoint of Carris API. During ingestion, transformations are applied directly to the data stream, such as cleaning, enrichment, and standardization, ensuring that only processed and structured data flows through the pipeline.

* Loading Transformed Data: The pre-processed data is then stored in a silver layer bucket on GCP. This layer serves as a structured repository, optimized for downstream analytical queries and consumption.

By prioritizing the ETL approach, this pipeline ensures that the data is transformed as it is ingested, minimizing the need for post-processing and enabling faster delivery of structured and actionable insights.



---



---



1. Authentication to Google Cloud Platform (GCP)


In [50]:
# autentication to gcloud with login

!gcloud auth application-default login


The environment variable [GOOGLE_APPLICATION_CREDENTIALS] is set to:
  [/content/.config/application_default_credentials.json]
Credentials will still be generated to the default location:
  [/content/.config/application_default_credentials.json]
To use these credentials, unset this environment variable before
running your application.

Do you want to continue (Y/n)?  n

ERROR: (gcloud.auth.application-default.login) Aborted by user.


In [3]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-23 19:21:58--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M   166MB/s    in 0.2s    

2025-01-23 19:21:58 (166 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]





---



---

2. Initialize SparkSession and set up the access to GCS


In [4]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')



---



---

3. Set up the source schema and initialize the readStream

In [30]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")



---



---

4. Transform

In [51]:
df_stops = spark.read.option("header", "true").csv('gs://edit-data-eng-project-group1/LandingZone/GTFS/stops.txt')
df_stops = df_stops.select('stop_id','stop_lat','stop_lon')
df_stops = df_stops.withColumn("stop_lat", df_stops["stop_lat"].cast("float"))
df_stops = df_stops.withColumn("stop_lon", df_stops["stop_lon"].cast("float"))

In [52]:
#select columns
transform = stream.select('id', 'timestamp','stop_id','lat', 'lon')
# join tables
transform = transform.join(df_stops, on='stop_id', how='left')


In [55]:
from pyspark.sql.functions import lag , col, coalesce, window
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
import math

# Define watermarking and window duration
watermark_duration = "120 seconds"
window_duration = "2 minutes"

# Using aggregate functions to get "last known" data within the window
windowed_transform = transform \
  .withWatermark("timestamp", watermark_duration) \
  .groupBy(F.window("timestamp", window_duration),"id") \
  .agg(
      F.max(col("timestamp")).alias("max_ts"),
      F.first("lat", True).alias("previous_lat"),
      F.first("lon", True).alias("previous_lon"),
      F.last("lat", True).alias("lat"),
      F.last("lon", True).alias("lon"),
      F.last("stop_lat", True).alias("stop_lat"),
      F.last("stop_lon", True).alias("stop_lon")
)


In [56]:
def haversine_distance(lat1, lon1, lat2, lon2):

    if any(x is None for x in [lat1, lon1, lat2, lon2]):
        return 0.0
    R = 6371  # Earth's radius in kilometers

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Calculate distance
    distance = R * c

    return distance

distance_udf = udf(haversine_distance, FloatType())

windowed_transform = windowed_transform.withColumn("distance_traveled", distance_udf(windowed_transform["previous_lat"],windowed_transform["previous_lon"],windowed_transform["lat"],windowed_transform["lon"]))
windowed_transform = windowed_transform.withColumn("distance_to_stop", distance_udf(windowed_transform["lat"],windowed_transform["lon"],windowed_transform["stop_lat"],windowed_transform["stop_lon"]))

In [58]:
agg = windowed_transform.withColumn('speed', col('distance_traveled')/(2/60))

agg = agg.filter(agg.distance_to_stop.isNotNull() & (agg.distance_to_stop > 0) & (agg.speed.isNotNull()) & (agg.speed > 0)) \
         .withColumn('time_to_stop', (col('distance_to_stop')/col('speed') * 3600))

agg = agg.withColumn(
    'time_to_stop',
    F.from_unixtime(
        F.unix_timestamp(F.lit('00:00:00'), 'HH:mm:ss') + col('time_to_stop'),
        'HH:mm:ss'
    ))



---



---


5. Load

In [ ]:
# Output function for each windowed batch
def insert_windowed_vehicles(df, batch_id):
    print(f"Batch ID: {batch_id} - Starting")
    try:
      df.write.format("parquet").mode("append").save("gs://edit-data-eng-project-group1/datalake/stream/ETL/")
      print(f"Batch ID: {batch_id} - Finished")
    except Exception as e:
      print(f"Batch ID: {batch_id} - Error: {e}")

# Write the streaming query with watermark and window
windowed_query = (agg
                  .writeStream
                  .outputMode("update")
                  .foreachBatch(insert_windowed_vehicles)
                  .option('checkpointLocation', 'gs://edit-data-eng-project-group1/datalake/stream/ETL/checkpoint')
                  .trigger(processingTime='31 seconds')
                  .start()
)

windowed_query.awaitTermination(200)

In [ ]:
windowed_query.isActive

In [ ]:
windowed_query.explain()

== Physical Plan ==
*(4) HashAggregate(keys=[window#5542, id#5560], functions=[max(timestamp#5570-T120000ms), last(lat#5561, true), last(lon#5563, true), first(lat#5561, true), first(lon#5563, true)])
+- StateStoreSave [window#5542, id#5560], state info [ checkpoint = gs://edit-data-eng-project-group1/datalake/stream/windowed_vehicles_1/checkpoint/state, runId = d6dca9f5-8bbe-43f0-b9bb-4d21c90f5506, opId = 0, ver = 0, numPartitions = 200], Update, 0, 0, 2
   +- *(3) HashAggregate(keys=[window#5542, id#5560], functions=[merge_max(timestamp#5570-T120000ms), merge_last(lat#5561, true), merge_last(lon#5563, true), merge_first(lat#5561, true), merge_first(lon#5563, true)])
      +- StateStoreRestore [window#5542, id#5560], state info [ checkpoint = gs://edit-data-eng-project-group1/datalake/stream/windowed_vehicles_1/checkpoint/state, runId = d6dca9f5-8bbe-43f0-b9bb-4d21c90f5506, opId = 0, ver = 0, numPartitions = 200], 2
         +- *(2) HashAggregate(keys=[window#5542, id#5560], functions

In [45]:
windowed_query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [ ]:
windowed_query.recentProgress

[]

In [46]:
windowed_query.stop()

Batch ID: 2 - Error: An error occurred while calling o946.save.
: java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1040)
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1345)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:242)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:258)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:187)
	at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:342)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:980)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2393)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeWrite$4(FileFormatWriter.scala:307)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.writeAndCommit

In [47]:
# Define the path to the Parquet files
parquet_path = "gs://edit-data-eng-project-group1/datalake/stream/windowed_vehicles_3"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()

+------------------------------------------+--------+-------------------+------------+------------+---------+---------+------------+--------------------+
|window                                    |id      |max_ts             |previous_lat|previous_lon|lat      |lon      |distance    |speed               |
+------------------------------------------+--------+-------------------+------------+------------+---------+---------+------------+--------------------+
|{2025-01-22 08:20:00, 2025-01-22 08:22:00}|44|12620|2025-01-22 08:21:54|38.622612   |-8.861655   |38.623722|-8.867623|0.53295755  |15.988726615905762  |
|{2025-01-22 08:20:00, 2025-01-22 08:22:00}|42|2313 |2025-01-22 08:21:38|38.804256   |-9.120041   |38.801754|-9.122209|0.33572677  |10.071803033351898  |
|{2025-01-22 08:20:00, 2025-01-22 08:22:00}|43|669  |2025-01-22 08:20:48|38.565853   |-9.039958   |38.565853|-9.039958|0.0         |0.0                 |
|{2025-01-22 08:20:00, 2025-01-22 08:22:00}|41|1390 |2025-01-22 08:21:26|38.

In [ ]:
from pyspark.sql.functions import min, max, col
parquet_df.agg(min(col('max_ts')), max(col('max_ts'))).show()

+-------------------+-------------------+
|        min(max_ts)|        max(max_ts)|
+-------------------+-------------------+
|2025-01-19 14:25:29|2025-01-21 17:22:22|
+-------------------+-------------------+



In [ ]:
# Define the path to the Parquet files
parquet_path = "gs://edit-data-eng-project-group1/datalake/stream/ETL"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()